In [2]:
import numpy as np
import os
import pandas as pd
import re

CNN_data="/home/asha/Documents/Bidirectiona-LSTM-for-text-summarization--master/cnn_stories/cnn/stories"
#daily_data="C:\\Users\\moseli\\Documents\\Masters of Information technology\\Masters project\\text mining\\data\\new CNN\\dailymail\\"
daily_data="/home/asha/Documents/Bidirectiona-LSTM-for-text-summarization--master/dailymail_stories/dailymail/stroies"
datasets={"cnn":CNN_data,"dailymail":daily_data}

data_categories=["training","validation","test"]

data={"articles":[],"summaries":[]}


def parsetext(dire,filename):
    with open("%s/%s"%(dire,filename),'r',encoding="Latin-1") as readin:
        print("file read successfully")
        text=readin.read()
    return text.lower()


def load_data(dire):
    """dataname refers to either training, test or validation"""
    filenames=[]
    for dirs,subdr, files in os.walk(dire):
        filenames=files
    return filenames


def cleantext(text):
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

def printArticlesum(k):
    print("---------------------original sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["articles"][k])
    print("----------------------Summary sentence-----------------------")
    print("-------------------------------------------------------------")
    print(data["summaries"][k])
    return 0


#def announcedone():
#    duration=2000
#    freq=440
#    ws.Beep(freq,duration)

filenames=load_data(datasets["cnn"])
# print(filenames)

"""----------load the data, sentences and summaries-----------"""
# print(filenames)
for k in range(len(filenames[:400])):
        if k%2==0:
            try:
                data["articles"].append(cleantext(parsetext(datasets["cnn"],"%s"%filenames[k])))
            except Exception as e:
                data["articles"].append("Could not read")
                print(e)
        else:
            try:
                data["summaries"].append(cleantext(parsetext(datasets["cnn"],"%s"%filenames[k])))
            except Exception as e:
                data["summaries"].append("Could not read")
                print(e)
print(data)
del filenames


file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read s

file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
file read successfully
{'articles': [' cnn     the year was  . it was the dawn of the soft drink machine, the first tube of th

In [3]:
from numpy.random import seed
seed(1)

# print(data.items())
modelLocation="/home/asha/Documents/Bidirectiona-LSTM-for-text-summarization--master/trainedmodel/"


import gensim as gs
import pandas as pd
import numpy as np
import scipy as sc
import nltk
from nltk.tokenize import word_tokenize as wt
from nltk.tokenize import sent_tokenize as st
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.sequence import pad_sequences
import logging
import re
from collections import Counter



logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

emb_size_all = 300
maxcorp=5000


def createCorpus(t):
    corpus = []
    all_sent = []
    for k in t:
        for p in t[k]:
            corpus.append(st(p))
    for sent in range(len(corpus)):
        for k in corpus[sent]:
            all_sent.append(k)
    for m in range(len(all_sent)):
        all_sent[m] = wt(all_sent[m])
    
    all_words=[]
    for sent in all_sent:
        hold=[]
        for word in sent:
            hold.append(word.lower())
        all_words.append(hold)
    return all_words


def word2vecmodel(corpus):
    emb_size = emb_size_all
    model_type={"skip_gram":1,"CBOW":0}
    window=10
    workers=4
    min_count=4
    batch_words=20
    epochs=25
    #include bigrams
    #bigramer = gs.models.Phrases(corpus)

    model=gs.models.Word2Vec(corpus,size=emb_size,sg=model_type["skip_gram"],
                             compute_loss=True,window=window,min_count=min_count,workers=workers,
                             batch_words=batch_words)
        
    model.train(corpus,total_examples=len(corpus),epochs=epochs)
    model.save("%sWord2vec"%modelLocation)
    print('\007')
    return model


def summonehot(corpus):
    allwords=[]
    annotated={}
    for sent in corpus:
        for word in wt(sent):
            allwords.append(word.lower())
    print(len(set(allwords)), "unique characters in corpus")
    #maxcorp=int(input("Enter desired number of vocabulary: "))
    maxcorp=int(len(set(allwords))/1.1)
    wordcount = Counter(allwords).most_common(maxcorp)
    allwords=[]
    
    for p in wordcount:
        allwords.append(p[0])  
        
    allwords=list(set(allwords))
    
    print(len(allwords), "unique characters in corpus after max corpus cut")
    #integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(allwords)
    #one hot
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #make look up dict
    for k in range(len(onehot_encoded)): 
        inverted = cleantext(label_encoder.inverse_transform([argmax(onehot_encoded[k, :])])[0]).strip()
        annotated[inverted]=onehot_encoded[k]
    return label_encoder,onehot_encoded,annotated



def wordvecmatrix(model,data):
    IO_data={"article":[],"summaries":[]}
    i=1
    for k in range(len(data["articles"])):
        art=[]
        summ=[]
        for word in wt(data["articles"][k].lower()):
            try:
                art.append(model.wv.word_vec(word))
            except Exception as e:
                print(e)

        for word in wt(data["summaries"][k].lower()):
            try:
                summ.append(onehot[word])
                #summ.append(model.wv.word_vec(word))
            except Exception as e:
                print(e)
        
        IO_data["article"].append(art) 
        IO_data["summaries"].append(summ)
        if i%100==0:
            print("progress: " + str(((i*100)/len(data["articles"]))))
        i+=1
    #announcedone()
    print('\007')
    return IO_data

def cutoffSequences(data,artLen,sumlen):
    data2={"article":[],"summaries":[]}
    for k in range(len(data["article"])):
        if len(data["article"][k])<artLen or len(data["summaries"][k])<sumlen:
             #data["article"]=np.delete(data["article"],k,0)
             #data["article"]=np.delete(data["summaries"],k,0)
             pass
        else:
            data2["article"].append(data["article"][k][:artLen])
            data2["summaries"].append(data["summaries"][k][:sumlen])
    return data2


def max_len(data):
    lenk=[]
    for k in data:
        lenk.append(len(k))
    print("The minimum length is: ",min(lenk))
    print("The average length is: ",np.average(lenk))
    print("The max length is: ",max(lenk))
    return min(lenk),max(lenk)

"""reshape vectres for Gensim"""
def reshape(vec):
    return np.reshape(vec,(1,emb_size_all))

def addones(seq):
    return np.insert(seq, [0], [[0],], axis = 0)

def endseq(seq):
    pp=len(seq)
    return np.insert(seq, [pp], [[1],], axis = 0)


corpus = createCorpus(data)

label_encoder,onehot_encoded,onehot=summonehot(data["summaries"])

model=word2vecmodel(corpus)

model.get_latest_training_loss()

train_data = wordvecmatrix(model,data)

print(len(train_data["article"]), "training articles")

train_data=cutoffSequences(train_data,300,25)

#seq length stats
max_len(train_data["article"])
max_len(train_data["summaries"])


train_data["summaries"]=np.array(train_data["summaries"])
train_data["article"]=np.array(train_data["article"])


#add end sequence for each article

#train_data["summaries"]=np.array(list(map(endseq,train_data["summaries"])))
#train_data["article"]=np.array(list(map(endseq,train_data["article"])))

print("summary length: ",len(train_data["summaries"][0]))
print("article length: ",len(train_data["article"][0]))


"""__pad sequences__
train_data["article"]=pad_sequences(train_data["article"],maxlen=max_len(train_data["article"]),
          padding='post',dtype=float)
train_data["summaries"]=pad_sequences(train_data["summaries"],maxlen=max_len(train_data["summaries"]),
          padding='post',dtype=float)
"""
#add start sequence
train_data["summaries"]=np.array(list(map(addones,train_data["summaries"])))



2020-05-24 15:06:47,165 : INFO : 'pattern' package not found; tag filters are not available for English


14212 unique characters in corpus
12919 unique characters in corpus after max corpus cut


/home/asha/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
2020-05-24 15:07:04,458 : INFO : collecting all words and their counts
2020-05-24 15:07:04,459 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-24 15:07:04,539 : INFO : PROGRESS: at sentence #10000, processed 237582 words, keeping 17592 word types
2020-05-24 15:07:04,568 : INFO : collected 20594 word types from a corpus of 319600 raw words and 13495 sentences
2020-05-24 15:

2020-05-24 15:07:36,123 : INFO : EPOCH 3 - PROGRESS: at 84.28% examples, 60044 words/s, in_qsize 7, out_qsize 2
2020-05-24 15:07:36,657 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-24 15:07:36,658 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-24 15:07:36,659 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-24 15:07:36,661 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-24 15:07:36,662 : INFO : EPOCH - 3 : training on 319600 raw words (213527 effective words) took 3.5s, 60322 effective words/s
2020-05-24 15:07:37,667 : INFO : EPOCH 4 - PROGRESS: at 28.31% examples, 61606 words/s, in_qsize 2, out_qsize 1
2020-05-24 15:07:38,668 : INFO : EPOCH 4 - PROGRESS: at 56.97% examples, 61580 words/s, in_qsize 8, out_qsize 0
2020-05-24 15:07:39,668 : INFO : EPOCH 4 - PROGRESS: at 86.55% examples, 61696 words/s, in_qsize 7, out_qsize 5
2020-05-24 15:07:40,154 : INFO : worker thread finishe

2020-05-24 15:08:10,337 : INFO : EPOCH 13 - PROGRESS: at 56.44% examples, 61152 words/s, in_qsize 8, out_qsize 3
2020-05-24 15:08:11,337 : INFO : EPOCH 13 - PROGRESS: at 85.87% examples, 61293 words/s, in_qsize 6, out_qsize 1
2020-05-24 15:08:11,813 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-05-24 15:08:11,815 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-05-24 15:08:11,816 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-05-24 15:08:11,817 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-05-24 15:08:11,818 : INFO : EPOCH - 13 : training on 319600 raw words (213843 effective words) took 3.5s, 61396 effective words/s
2020-05-24 15:08:12,824 : INFO : EPOCH 14 - PROGRESS: at 28.52% examples, 62061 words/s, in_qsize 7, out_qsize 4
2020-05-24 15:08:13,825 : INFO : EPOCH 14 - PROGRESS: at 57.41% examples, 62094 words/s, in_qsize 1, out_qsize 2
2020-05-24 15:08:14,825 : INFO : EPOCH 14 - PROGR

...

"word 'invalidate' not in vocabulary"
"word 'andres' not in vocabulary"
"word 'presumptive' not in vocabulary"
"word 'enrique' not in vocabulary"
"word 'nieto' not in vocabulary"
"word 'accusation' not in vocabulary"
"word 'recount' not in vocabulary"
"word 'erase' not in vocabulary"
"word 'nieto' not in vocabulary"
"word 'institutional' not in vocabulary"
"word 'tribunal' not in vocabulary"
"word 'ratifies' not in vocabulary"
"word 'tribunal' not in vocabulary"
"word 'ratify' not in vocabulary"
"word 'concede' not in vocabulary"
"word 'leftist' not in vocabulary"
"word 'felipe' not in vocabulary"
"word 'calderon' not in vocabulary"
"word 'ins' not in vocabulary"
"word 'blockades' not in vocabulary"
"word 'scanned' not in vocabulary"
"word 'copies' not in vocabulary"
"word 'arisen' not in vocabulary"
"word 'manipulation' not in vocabulary"
"word 'redeemed' not in vocabulary"
"word 'chain' not in vocabulary"
"word 'supermarkets' not in vocabulary"
"word 'farce' not in vocabulary"
"word 

'bernadine'
"'oh"
'cardiologist'
'institutes'
"'yes"
'jokes'
'clots'
'pleading'
'wulf'
'obstetrics'
'gynecology'
'hormonal'
'alleviate'
'layering'
'yoga'
'marcia'
'sleeplessness'
'sleepless'
'lactose'
'intolerant'
'tolerated'
'shortest'
'fuzzy'
'pluses'
'minuses'
'importance'
'internist'
'obstetricians'
'sabriya'
'medically'
"word 'galleon' not in vocabulary"
"word 'steal' not in vocabulary"
"word 'rajarantnam' not in vocabulary"
"word 'wharton' not in vocabulary"
"word 'illicit' not in vocabulary"
"word 'wiretaps' not in vocabulary"
"word 'securities' not in vocabulary"
"word 'conspired' not in vocabulary"
"word 'galleon' not in vocabulary"
"word 'consultants' not in vocabulary"
"word 'seated' not in vocabulary"
"word 'pled' not in vocabulary"
"word 'holwells' not in vocabulary"
"word 'overflowing' not in vocabulary"
"word 'overflow' not in vocabulary"
"word 'dowd' not in vocabulary"
"word 'painstakingly' not in vocabulary"
"word 'traded' not in vocabulary"
"word 'dowd' not in vocabul

'artyem'
'psychotic'
'spiderman'
'grandma'
'copy'
'dme'
'escorted'
'envelopes'
'tverskaya'
'receptionist'
'deceived'
'pencils'
'logan'
'gifts'
'handed'
'threatens'
'consular'
'straw'
'psychologist'
'communicative'
'galina'
'semya'
'aftereffects'
'trauma'
'randall'
'eal'
'crawford'
'constitutes'
"word 'downed' not in vocabulary"
"word 'breaches' not in vocabulary"
"word 'intervening' not in vocabulary"
"word 'needy' not in vocabulary"
"word 'dire' not in vocabulary"
"word 'hassam' not in vocabulary"
"word 'zaki' not in vocabulary"
"word 'spree' not in vocabulary"
"word 'zaki' not in vocabulary"
"word 'aryeh' not in vocabulary"
"word 'mekel' not in vocabulary"
"word 'exodus' not in vocabulary"
"word 'gazas' not in vocabulary"
"word 'mindful' not in vocabulary"
"word 'pact' not in vocabulary"
"word 'tersely' not in vocabulary"
"word 'worded' not in vocabulary"
"word 'operates' not in vocabulary"
"word 'accordance' not in vocabulary"
"word 'egyptians' not in vocabulary"
"word 'cheered' not

"word 'updated' not in vocabulary"
"word 'provoke' not in vocabulary"
"word 'brushfires' not in vocabulary"
"word 'cyclone' not in vocabulary"
"word 'tegan' not in vocabulary"
"word 'sapwell' not in vocabulary"
"word 'tegan' not in vocabulary"
"word 'tegan' not in vocabulary"
"word 'sapwell' not in vocabulary"
"word 'touches' not in vocabulary"
"word 'shrugged' not in vocabulary"
"word 'rebuild' not in vocabulary"
"word 'aussie' not in vocabulary"
"word 'windy' not in vocabulary"
"word 'arson' not in vocabulary"
"word 'lisk' not in vocabulary"
"word 'foreclosure' not in vocabulary"
"word 'foreclosures' not in vocabulary"
"word 'foreclosure' not in vocabulary"
"word 'homeowner' not in vocabulary"
"word 'foreclosures' not in vocabulary"
"word 'foreclosed' not in vocabulary"
"word 'foreclosed' not in vocabulary"
"word 'renovations' not in vocabulary"
"word 'redevelop' not in vocabulary"
"word 'bing' not in vocabulary"
"word 'deter' not in vocabulary"
"word 'toyotas' not in vocabulary"
"wo

'objectives'
progress: 100.0

200 training articles
The minimum length is:  300
The average length is:  300.0
The max length is:  300
The minimum length is:  25
The average length is:  25.0
The max length is:  25
summary length:  25
article length:  300


In [4]:
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split as tts
import logging

from pyrouge import Rouge155 
import matplotlib.pyplot as plt
import keras
from keras import backend as k
k.set_learning_phase(1)
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import Model
from keras.layers import Dense,LSTM,Input,Activation,Add,TimeDistributed,\
Permute,Flatten,RepeatVector,merge,Lambda,Multiply,Reshape
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)
num_classes = 1
epochs = 20
hidden_units = emb_size_all
learning_rate = 0.005
clip_norm = 2.0

en_shape=np.shape(train_data["article"][0])
print(en_shape[1])
de_shape=np.shape(train_data["summaries"][0])

def encoder_decoder(data):
    print('Encoder_Decoder LSTM...')
   
    """__encoder___"""
    encoder_inputs = Input(shape=(en_shape,))
    print(encoder_inputs)
    
    encoder_LSTM = LSTM(hidden_units,dropout_U=0.2,dropout_W=0.2,return_sequences=True,return_state=True)
    encoder_LSTM_rev=LSTM(hidden_units,return_state=True,return_sequences=True,dropout_U=0.05,dropout_W=0.05,go_backwards=True)
    
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)
    encoder_outputsR, state_hR, state_cR = encoder_LSTM_rev(encoder_inputs)
    
    state_hfinal=Add()([state_h,state_hR])
    state_cfinal=Add()([state_c,state_cR])
    encoder_outputs_final = Add()([encoder_outputs,encoder_outputsR])
    
    encoder_states = [state_hfinal,state_cfinal]
    
    """____decoder___"""
    decoder_inputs = Input(shape=(None,de_shape[1]))
    decoder_LSTM = LSTM(hidden_units,return_sequences=True,dropout_U=0.2,dropout_W=0.2,return_state=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_inputs,initial_state=encoder_states)
    
    #Pull out XGBoost, (I mean attention)
    attention = TimeDistributed(Dense(1, activation = 'tanh'))(encoder_outputs_final)
    attention = Flatten()(attention)
    attention = Multiply()([decoder_outputs, attention])
    attention = Activation('softmax')(attention)
    attention = Permute([2, 1])(attention)
 
    decoder_dense = Dense(de_shape[1],activation='softmax')
    decoder_outputs = decoder_dense(attention)
    
    model= Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)
    print(model.summary())
    
    rmsprop = RMSprop(lr=learning_rate,clipnorm=clip_norm)
    model.compile(loss='categorical_crossentropy',optimizer=rmsprop,metrics=['accuracy'])
    
    x_train,x_test,y_train,y_test=tts(data["article"],data["summaries"],test_size=0.20)
    history= model.fit(x=[x_train,y_train],
              y=y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=([x_test,y_test], y_test))
    print(model.summary())
    """_________________inference mode__________________"""
    encoder_model_inf = Model(encoder_inputs,encoder_states)
    
    decoder_state_input_H = Input(shape=(en_shape[0],))
    decoder_state_input_C = Input(shape=(en_shape[0],)) 
    decoder_state_inputs = [decoder_state_input_H, decoder_state_input_C]
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_LSTM(decoder_inputs,
                                                                     initial_state=decoder_state_inputs)
    decoder_states = [decoder_state_h, decoder_state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model_inf= Model([decoder_inputs]+decoder_state_inputs,
                         [decoder_outputs]+decoder_states)
    
    scores = model.evaluate([x_test,y_test],y_test, verbose=1)
    
    
    print('LSTM test scores:', scores)
    print('\007')
    print(model.summary())
    return model,encoder_model_inf,decoder_model_inf,history


"""_________generate summary from vectors_____________"""


def generateText(SentOfVecs):
    SentOfVecs=np.reshape(SentOfVecs,de_shape)
    kk=""
    for k in SentOfVecs:
        kk = kk + label_encoder.inverse_transform([argmax(k)])[0].strip()+" "
        #kk=kk+((getWord(k)[0]+" ") if getWord(k)[1]>0.01 else "")
    return kk

"""________________generate summary vectors___________"""

def summarize(article):
    stop_pred = False
    article =  np.reshape(article,(1,en_shape[0],en_shape[1]))
    #get initial h and c values from encoder
    init_state_val = encoder.predict(article)
    target_seq = np.zeros((1,1,de_shape[1]))
    #target_seq =np.reshape(train_data['summaries'][k][0],(1,1,de_shape[1]))
    generated_summary=[]
    while not stop_pred:
        decoder_out,decoder_h,decoder_c= decoder.predict(x=[target_seq]+init_state_val)
        generated_summary.append(decoder_out)
        init_state_val= [decoder_h,decoder_c]
        #get most similar word and put in line to be input in next timestep
        #target_seq=np.reshape(model.wv[getWord(decoder_out)[0]],(1,1,emb_size_all))
        target_seq=np.reshape(decoder_out,(1,1,de_shape[1]))
        if len(generated_summary)== de_shape[0]:
            stop_pred=True
            break
    return generated_summary

"""__________________Plot training curves_______________"""

def plot_training(history):
    print(history.history.keys())
    #  "Accuracy"
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # "Loss"
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


def saveModels():
    trained_model.save("%sinit_model"%modelLocation)
    encoder.save("%sencoder"%modelLocation)
    decoder.save("%sdecoder"%modelLocation)
    
def evaluate_summ(article):
    ref=''
    for k in wt(data['summaries'][article])[:20]:
        ref=ref+' '+k
    gen_sum = generateText(summarize(train_data["article"][article]))
    print("-----------------------------------------------------")
    print("Original summary")
    print(ref)
    print("-----------------------------------------------------")
    print("Generated summary")
    print(gen_sum)
    print("-----------------------------------------------------")
    rouge = Rouge155()
    score = rouge.score_summary(ref, gen_sum)
    print("Rouge1 Score: ",score)
        
#######################################################################################
################################ Train model and test##################################
#######################################################################################

trained_model,encoder,decoder,history = encoder_decoder(train_data)
plot_training(history)
evaluate_summ(10)

print(generateText(summarize(train_data["article"][8])))
print(data["summaries"][8])
print(data["articles"][78])



300
Encoder_Decoder LSTM...


TypeError: Error converting shape to a TensorShape: Dimension value must be integer or None or have an __index__ method, got (300, 300).

In [29]:
!pip install pyrouge

    100% |████████████████████████████████| 61kB 95kB/s ta 0:00:011
  Stored in directory: /home/asha/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


In [50]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
	X1, X2, y = list(), list(), list()
	for _ in range(n_samples):
		# generate source sequence
		source = generate_sequence(n_in, cardinality)
		# define padded target sequence
		target = source[:n_out]
		target.reverse()
		# create padded input target sequence
		target_in = [0] + target[:-1]
		# encode
		src_encoded = to_categorical([source], num_classes=cardinality)
		tar_encoded = to_categorical([target], num_classes=cardinality)
		tar2_encoded = to_categorical([target_in], num_classes=cardinality)
		# store
		X1.append(src_encoded)
		X2.append(tar2_encoded)
		y.append(tar_encoded)
	return array(X1), array(X2), array(y)

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape,X2.shape,y.shape)
# train model
train.fit([X1, X2], y, epochs=1)
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

TypeError: Expected Operation, Variable, or Tensor, got 1

In [60]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  3 09:31:45 2018

@author: moseli
"""
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split as tts
import logging

import chart_studio.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import pydot


import keras
from keras import backend as k
k.set_learning_phase(1)
from keras.preprocessing.text import Tokenizer
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,Concatenate
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam
#keras.utils.vis_utils import plot_model


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

#######################model params###########################
batch_size = 100
num_classes = 1
epochs = 5
hidden_units = emb_size_all
learning_rate = 0.002
clip_norm = 1.0

###############################################################
en_shape=np.shape(train_data["article"][0])
de_shape=np.shape(train_data["summaries"][0])

"""______generate summary length for test______"""
#train_data["nums_summ"]=list(map(lambda x:0 if len(x)<5000 else 1,data["articles"]))
#train_data["nums_summ"]=list(map(len,data["summaries"]))
#train_data["nums_summ_norm"]=(np.array(train_data["nums_summ"])-min(train_data["nums_summ"]))/(max(train_data["nums_summ"])-min(train_data["nums_summ"]))

############################helpers###########################

def encoder_decoder(data):
    print('Encoder_Decoder LSTM...')
   
    """__encoder___"""
    encoder_inputs = Input(shape=en_shape)
    
    encoder_LSTM = LSTM(hidden_units, dropout_U = 0.2, dropout_W = 0.2 ,return_state=True)
    encoder_LSTM_rev=LSTM(hidden_units,return_state=True,go_backwards=True)
    
    #merger=Add()[encoder_LSTM(encoder_inputs), encoder_LSTM_rev(encoder_inputs)]
    encoder_outputsR, state_hR, state_cR = encoder_LSTM_rev(encoder_inputs)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)
    
    state_hfinal=Add()([state_h,state_hR])
    state_cfinal=Add()([state_c,state_cR])
    
    encoder_states = [state_hfinal,state_cfinal]
    
    """____decoder___"""
    decoder_inputs = Input(shape=(None,de_shape[1]))
    decoder_LSTM = LSTM(hidden_units,return_sequences=True,return_state=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_inputs,initial_state=encoder_states) 
    decoder_dense = Dense(de_shape[1],activation='linear')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model= Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)
    #plot_model(model, to_file=modelLocation+'model.png', show_shapes=True)
    rmsprop = RMSprop(lr=learning_rate,clipnorm=clip_norm)
    
    model.compile(loss='mse',optimizer=rmsprop)

    x_train,x_test,y_train,y_test=tts(data["article"],data["summaries"],test_size=0.20)
    model.fit(x=[x_train,y_train],
              y=y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=([x_test,y_test], y_test))
    
    """_________________inference mode__________________"""
    encoder_model_inf = Model(encoder_inputs,encoder_states)
    
    decoder_state_input_H = Input(shape=(hidden_units,))
    decoder_state_input_C = Input(shape=(hidden_units,)) 
    decoder_state_inputs = [decoder_state_input_H, decoder_state_input_C]
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_LSTM(decoder_inputs,
                                                                     initial_state=decoder_state_inputs)
    decoder_states = [decoder_state_h, decoder_state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model_inf= Model([decoder_inputs]+decoder_state_inputs,
                         [decoder_outputs]+decoder_states)
    
    
    #plot_model(encoder_model_inf, to_file='encoder_model.png', show_shapes=True)
    #plot_model(decoder_model_inf, to_file='decoder_model.png', show_shapes=True)
    scores = model.evaluate([x_test,y_test],y_test, verbose=0)
    
    
    print('LSTM test scores:', scores)
    #announcedone()
    print('\007')
    print(model.summary())
    return model,encoder_model_inf,decoder_model_inf

"""___pred____"""
def comparePred(index):
    pred=trained_model.predict([np.reshape(train_data["article"][index],(1,en_shape[0],emb_size_all)),np.reshape(train_data["summaries"][index],(1,de_shape[0],emb_size_all))])
    return pred


"""____generate summary from vectors and remove padding words___"""
def generateText(SentOfVecs):
    SentOfVecs=np.reshape(SentOfVecs,de_shape)
    kk=""
    for k in SentOfVecs:
        kk=kk+((getWord(k)[0]+" ") if getWord(k)[1]>0.2 else "")
    return kk

"""___generate summary vectors___"""

def summarize(article):
    stop_pred = False
    article =  np.reshape(article,(1,en_shape[0],en_shape[1]))
    
    #get initial h and c values from encoder
    init_state_val = encoder.predict(article)
    target_seq = np.zeros((1,1,emb_size_all))
    
    generated_summary=[]
    while not stop_pred:
        decoder_out,decoder_h,decoder_c= decoder.predict(x=[target_seq]+init_state_val)
        generated_summary.append(decoder_out)
        init_state_val= [decoder_h,decoder_c]
        #get most similar word and put in line to be input in next timestep
        #target_seq=np.reshape(model.wv[getWord(decoder_out)[0]],(1,1,emb_size_all))
        target_seq=np.reshape(decoder_out,(1,1,emb_size_all))
        if len(generated_summary)== de_shape[0]:
            stop_pred=True
            break
    return generated_summary
        
        
#######################################################################################

trained_model,encoder,decoder = encoder_decoder(train_data)

def saveModels():
    trained_model.save("%sinit_model"%modelLocation)
    encoder.save("%sencoder"%modelLocation)
    decoder.save("%sdecoder"%modelLocation)

print(generateText(summarize(train_data["article"][10])))
print(data["summaries"][10])
print(data["articles"][10])

del trained_model,encoder,decoder


getWord(collect_pred[23])
model.wv.most_similar(np.zeros((1,emb_size_all)))

Encoder_Decoder LSTM...


/home/asha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, return_state=True, dropout=0.2, recurrent_dropout=0.2)`


TypeError: Expected Operation, Variable, or Tensor, got 1

In [59]:
!pip install pydot